In [9]:
import os
os.environ["MLFLOW_TRACKING_URI"] = 'https://dagshub.com/matheusbeuren/data-science-project.mlflow'
os.environ["MLFLOW_TRACKING_USERNAME"] = 'matheusbeuren'
os.environ["MLFLOW_TRACKING_PASSWORD"] = '281f77ce3fe6c2c42bafada060957e98b3ecdbf5'

In [10]:
import os
os.chdir('C:/Users/mathe/Udemy/Mlops_course/section18/data-science-project')
%pwd

'C:\\Users\\mathe\\Udemy\\Mlops_course\\section18\\data-science-project'

In [11]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model: Path
    test_file_path: Path
    metric_file_path: Path
    all_params: dict
    target_col: str
    mlflow_uri: str

In [12]:
from src.DataScienceProject.utils.common import read_yaml, create_directories, save_json
from src.DataScienceProject.constants import *



In [13]:
class ConfigurationManager:
    def __init__(
        self,
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params= self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])    

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model = config.model,
            test_file_path = config.test_file_path,
            metric_file_path = config.metric_file_path,
            all_params = self.params,
            target_col = self.schema.TARGET_COLUMN,
            mlflow_uri = os.environ["MLFLOW_TRACKING_URI"]

        )

        return model_evaluation_config

In [14]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [15]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def evaluate_metrics(self, actual, predicted):
        r2= np.sqrt(mean_squared_error(actual,predicted))
        mae= mean_absolute_error(actual,predicted)
        mse= r2_score(actual,predicted)
        return r2, mae, mse
    
    def evaluate(self):
        test_data = pd.read_csv(self.config.test_file_path)
        model = joblib.load(self.config.model)

        test_x = test_data.drop(self.config.target_col, axis=1)
        test_y = test_data[self.config.target_col]

        mlflow.set_tracking_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            predicted = model.predict(test_x)
            (r2, mae, mse) = self.evaluate_metrics(test_y, predicted)

            score = {
                "r2_score": r2,
                "mean_absolute_error": mae,
                "mean_squared_error": mse
            }
            save_json(path=Path(self.config.metric_file_path), data=score)

            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(score)
            

            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [16]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.evaluate()
except Exception as e:
    raise e

[2026-02-06 07:56:01,583: INFO: common: YAML file: config\config.yaml carregado com sucesso:]
[2026-02-06 07:56:01,590: INFO: common: YAML file: params.yaml carregado com sucesso:]
[2026-02-06 07:56:01,595: INFO: common: YAML file: schema.yaml carregado com sucesso:]
[2026-02-06 07:56:01,595: INFO: common: Diretório criado em: artifacts:]
[2026-02-06 07:56:01,599: INFO: common: Diretório criado em: artifacts/model_evaluation:]
[2026-02-06 07:56:01,998: INFO: common: JSON salvo em: artifacts\model_evaluation\metric.json:]


2026/02/06 07:56:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
c:\Users\mathe\Udemy\Mlops_course\section18\data-science-project\venv\lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended safe alternative is the 'skops' format.
  flavor.save_model(path=local_path, mlflow_model=mlflow_model, **kwargs)
Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2026/02/06 07:56:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticNetModel, version 2
Created version '2' of model 'ElasticNetModel'.


🏃 View run smiling-pug-518 at: https://dagshub.com/matheusbeuren/data-science-project.mlflow/#/experiments/0/runs/f3e7ca9391ba4071b0a55a30e2243fa9
🧪 View experiment at: https://dagshub.com/matheusbeuren/data-science-project.mlflow/#/experiments/0
